In [1]:
import os
import sys
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit
import scipy.odr as odr

In [2]:
#load filter-based absorption photometer data
Filter_based_instrument_data  = pd.read_csv("C:/Users/cphal/Desktop/MAC prediction model/Codes for the MAC prediction model/Input datasets/Module B/Babs.csv")
Filter_based_instrument_data['datetime_Babs'] = pd.to_datetime(Filter_based_instrument_data['datetime_Babs'])

#load Bscat data
Bscat_data  = pd.read_csv("C:/Users/cphal/Desktop/MAC prediction model/Codes for the MAC prediction model/Input datasets/Module B/Bscat.csv")

In [3]:
datetime= Filter_based_instrument_data['datetime_Babs'] 
batn_red=Filter_based_instrument_data['batn_red']
batn_green=Filter_based_instrument_data['batn_green']
batn_blue=Filter_based_instrument_data['batn_blue']
Tr_red=Filter_based_instrument_data['Tr_red']
Tr_green=Filter_based_instrument_data['Tr_green']
Tr_blue=Filter_based_instrument_data['Tr_blue']
bscat_red=Bscat_data['b_scat_red']
bscat_green=Bscat_data['b_scat_green']
bscat_blue=Bscat_data['b_scat_blue']

In [4]:
def AAE_fit_one_measurement( wavelength, AAE_fit, AAE_constant):
    return  AAE_constant*np.power(wavelength, -AAE_fit)

In [5]:
def AAE_power_fit(babs_wvl_red,babs_wvl_green,babs_wvl_blue,w1,w2,w3):

    wavelength_instrument=np.array([w1,w2,w3])  # modify if necessary

    temp_AAE = np.nan; temp_AAE_constant = np.nan
    temp_sigma = np.nan; temp_sigma_constant = np.nan  #uncertainty of fit parameters

    AAE_three_lambda = np.empty(len(babs_wvl_red)); AAE_constant = np.empty(len(babs_wvl_red))
    sigma_AAE = np.empty(len(babs_wvl_red)); sigma_constant = np.empty(len(babs_wvl_red))

    #compute AAE_two_lambda, which are used later when initializing AAE_three_lambda
    AAE_R_G = -np.log(babs_wvl_red/babs_wvl_green)/np.log(wavelength_instrument[0]/wavelength_instrument[1])
    AAE_R_B = -np.log(babs_wvl_red/babs_wvl_blue)/np.log(wavelength_instrument[0]/wavelength_instrument[2])
    AAE_G_B = -np.log(babs_wvl_green/babs_wvl_blue)/np.log(wavelength_instrument[1]/wavelength_instrument[2])

    for m in range(len(babs_wvl_red)):
        if (np.isnan(babs_wvl_red[m])==0 and np.isnan(babs_wvl_green[m])==0 and np.isnan(babs_wvl_blue[m])==0):

            one_babs_at_three_lambda=[babs_wvl_red[m],babs_wvl_green[m],babs_wvl_blue[m]]
                
            #use AAE_G_B as initial input
            W_coef = np.array([AAE_G_B.values[m], babs_wvl_green[m]/ (np.power(wavelength_instrument[1], -AAE_G_B.values[m]))] )
            try:
                W_coef, w_sigma = curve_fit(AAE_fit_one_measurement, wavelength_instrument, one_babs_at_three_lambda,p0=W_coef, maxfev=1000)
                w_sigma = np.sqrt(np.diag(w_sigma))
            except RuntimeError:
                pass
            temp_AAE = W_coef[0];temp_AAE_constant = W_coef[1];temp_sigma= w_sigma[0];temp_sigma_constant = w_sigma[1]
            #use AAE_R_G as initial input
            W_coef = np.array([AAE_R_G.values[m], babs_wvl_red[m]/ (np.power(wavelength_instrument[0], -AAE_R_G.values[m]))]  ) 
            try:          
                W_coef, w_sigma = curve_fit(AAE_fit_one_measurement, wavelength_instrument, one_babs_at_three_lambda,p0=W_coef, maxfev=1000)
                w_sigma = np.sqrt(np.diag(w_sigma))
            except RuntimeError:
                pass
            
            #check if the current inital guess (AAE_R_G) is better than the previous one (AAE_G_B): ? w_sigma is smaller
            if (w_sigma[0]<temp_sigma):  #Yes, the current w_sigma is smaller                
                temp_AAE = W_coef[0];temp_AAE_constant = W_coef[1];temp_sigma= w_sigma[0];temp_sigma_constant = w_sigma[1]
            else:
                pass       #do not replace the previous results
            
            #use AAE_R_B as initial input
            W_coef = np.array([AAE_R_B.values[m],babs_wvl_blue[m]/ (np.power(wavelength_instrument[2], -AAE_R_B.values[m]))])
            try:   
                W_coef, w_sigma = curve_fit(AAE_fit_one_measurement, wavelength_instrument, one_babs_at_three_lambda,p0=W_coef, maxfev=1000)
                w_sigma = np.sqrt(np.diag(w_sigma))
            except RuntimeError:
                pass               
            #check if the current initial guess (AAE_R_B) is better than the previous one (AAE_G_B or AAE_R_G): ? w_sigma is smaller
            if (w_sigma[0]<temp_sigma):  #Yes, the current w_sigma is smaller                        
                temp_AAE = W_coef[0];temp_AAE_constant = W_coef[1];temp_sigma= w_sigma[0];temp_sigma_constant = w_sigma[1]
            else: 
                pass #do not replace the previous results   
            
        #Output the results
        AAE_three_lambda[m]=temp_AAE; AAE_constant[m]=temp_AAE_constant
        sigma_AAE[m]=temp_sigma; sigma_constant[m]=temp_sigma_constant
        #clear temp_variables
        temp_AAE=np.nan;temp_AAE_constant=np.nan
        temp_sigma=np.nan;temp_sigma_constant=np.nan

    return AAE_three_lambda

In [6]:
def AAE_fit_one_measurement( wavelength, AAE_fit, AAE_constant):
    return  AAE_constant*np.power(wavelength, -AAE_fit)

In [7]:
def Algorithm_function(X,w0,w1,w2,w3,w4,w5,w6,w7):
    ln_tr_in_function, SSA_in_function, AAE_in_function = X
    return w0+w1*ln_tr_in_function+w2*SSA_in_function+w3*AAE_in_function+\
        w4*ln_tr_in_function*SSA_in_function+w5*SSA_in_function*AAE_in_function+\
            w6*ln_tr_in_function*AAE_in_function+w7*ln_tr_in_function*AAE_in_function*SSA_in_function


In [8]:
def linear_func(p, x):
   m, c = p
   return m*x + c

In [10]:
def correct_Babs(bscat_red,bscat_green,bscat_blue,batn_red,batn_green,batn_blue,Tr_red,Tr_green,Tr_blue, w1, w2, w3, algorithm = "L2020"):
    #initialize AAE, using batn
    AAE_three_lambda = AAE_power_fit(batn_red,batn_green,batn_blue,w1,w2,w3)
    AAE_temp =  pd.Series(AAE_three_lambda)

    AAE_temp[AAE_temp < 0 ] = 0;    AAE_temp[AAE_temp > 10 ] = 0
    batn_red[np.isnan(AAE_temp)] = 0
    #initialize SSA, using batn and bscat_ref
    SSA_red_temp=bscat_red/(bscat_red+batn_red)
    SSA_green_temp=bscat_green/(bscat_green+batn_green)
    SSA_blue_temp=bscat_blue/(bscat_blue+batn_blue)
    batn_red[SSA_red_temp < 0 ] = np.nan;     batn_red[SSA_red_temp >1 ] = np.nan
    #compute ln_Tr
    ln_Tr_red=np.log(Tr_red)
    ln_Tr_green=np.log(Tr_green)
    ln_Tr_blue=np.log(Tr_blue)
   
    #Initialize set of coefficients. SGP_CLAP coefficients are used in the current code.

    #Recommended initial guesses for the other combinations of 
    #aerosol type and filter-based absorption photometer can be found in Li et al (2020).
    W_coef_temp_red= np.array([0.239035,0.34574,-0.157059,-0.0433201,-0.468967,0.0703361,-0.565455,0.727888])
    W_coef_temp_green= np.array([0.300341,0.481262,-0.260768,-0.100724,-0.660501,0.168658,-0.63395,0.790648])
    W_coef_temp_blue= np.array([0.353833,0.492597,-0.335853,-0.148951,-0.685201,0.232535,-0.552151,0.687165])
    
    #Calculate temporary Babs for each wavelength using Eq. (8) in Li et al (2020)
    b_abs_red_temp=batn_red*(W_coef_temp_red[0]+W_coef_temp_red[1]*ln_Tr_red+W_coef_temp_red[2]*SSA_red_temp+W_coef_temp_red[3]*AAE_temp+W_coef_temp_red[4]*ln_Tr_red*SSA_red_temp+W_coef_temp_red[5]*SSA_red_temp*AAE_temp+W_coef_temp_red[6]*ln_Tr_red*AAE_temp+W_coef_temp_red[7]*ln_Tr_red*AAE_temp*SSA_red_temp)
    b_abs_green_temp=batn_green*(W_coef_temp_green[0]+W_coef_temp_green[1]*ln_Tr_green+W_coef_temp_green[2]*SSA_green_temp+W_coef_temp_green[3]*AAE_temp+W_coef_temp_green[4]*ln_Tr_green*SSA_green_temp+W_coef_temp_green[5]*SSA_green_temp*AAE_temp+W_coef_temp_green[6]*ln_Tr_green*AAE_temp+W_coef_temp_green[7]*ln_Tr_green*AAE_temp*SSA_green_temp)
    b_abs_blue_temp=batn_blue*(W_coef_temp_blue[0]+W_coef_temp_blue[1]*ln_Tr_blue+W_coef_temp_blue[2]*SSA_blue_temp+W_coef_temp_blue[3]*AAE_temp+W_coef_temp_blue[4]*ln_Tr_blue*SSA_blue_temp+W_coef_temp_blue[5]*SSA_blue_temp*AAE_temp+W_coef_temp_blue[6]*ln_Tr_blue*AAE_temp+W_coef_temp_blue[7]*ln_Tr_blue*AAE_temp*SSA_blue_temp)

    #compute "g" term in Eq. (9) of Li et al (2020), where g(Tr,SSA,AAE)=babs_ref/batn
    g_term_red_temp=b_abs_red_temp/batn_red
    g_term_green_temp=b_abs_green_temp/batn_green
    g_term_blue_temp=b_abs_blue_temp/batn_blue

    batn_red[g_term_red_temp < 0.001 ] =  np.nan; batn_red[g_term_red_temp >1] =  np.nan
    batn_green[np.isnan(batn_red)] = 0; batn_blue[np.isnan(batn_red)] = 0
    ln_Tr_red[np.isnan(batn_red)] = 0
    ln_Tr_green[np.isnan(batn_red)] = 0
    ln_Tr_blue[np.isnan(batn_red)] = 0

    loop_number=0
    W_coef =[10,10] #random numbers, used for initialization
    print ("////////////iterative process////////////")
    while W_coef[0]>1.015 or W_coef[0]<0.985:

        loop_number += 1
        print ("Loop No.", loop_number)

        #temporarily save the outputs from the previous iteration
        AAE_previous_cycle = AAE_temp 

        W_coef = W_coef_temp_red
        valid = ~(np.isnan(ln_Tr_red) | np.isnan(SSA_red_temp)|np.isnan(AAE_temp)| np.isnan(g_term_red_temp))
        W_coef, w_sigma = curve_fit(Algorithm_function, (ln_Tr_red[valid], SSA_red_temp[valid], AAE_temp[valid]), g_term_red_temp[valid], p0=W_coef, maxfev=1000)
        W_coef_temp_red  = W_coef

        W_coef =W_coef_temp_green 
        valid = ~(np.isnan(ln_Tr_green) | np.isnan(SSA_green_temp)|np.isnan(AAE_temp)| np.isnan(g_term_green_temp))
        W_coef, w_sigma = curve_fit(Algorithm_function, (ln_Tr_green[valid],SSA_green_temp[valid],AAE_temp[valid]), g_term_green_temp[valid],p0=W_coef, maxfev=1000)
        W_coef_temp_green = W_coef 

        W_coef =W_coef_temp_blue[:] 
        valid = ~(np.isnan(ln_Tr_blue) | np.isnan(SSA_blue_temp)|np.isnan(AAE_temp)| np.isnan(g_term_blue_temp))
        W_coef, w_sigma = curve_fit(Algorithm_function, (ln_Tr_blue[valid],SSA_blue_temp[valid],AAE_temp[valid]), g_term_blue_temp[valid],p0=W_coef, maxfev=1000)
        W_coef_temp_blue=W_coef

        print ("W_coef_temp_red",W_coef_temp_red)
        print ("W_coef_temp_green",W_coef_temp_green)
        print ("W_coef_temp_blue",W_coef_temp_blue)

        #Update Babs for each wavelength using Eq. (8) in Li et al (2020)
        b_abs_red_temp = batn_red*(W_coef_temp_red[0]+W_coef_temp_red[1]*ln_Tr_red+W_coef_temp_red[2]*SSA_red_temp+W_coef_temp_red[3]*AAE_temp+W_coef_temp_red[4]*ln_Tr_red*SSA_red_temp +W_coef_temp_red[5]*SSA_red_temp*AAE_temp+W_coef_temp_red[6]*ln_Tr_red*AAE_temp+W_coef_temp_red[7]*ln_Tr_red*AAE_temp*SSA_red_temp)

        b_abs_green_temp = batn_green*(W_coef_temp_green[0]+W_coef_temp_green[1]*ln_Tr_green+W_coef_temp_green[2]*SSA_green_temp+W_coef_temp_green[3]*AAE_temp+W_coef_temp_green[4]*ln_Tr_green*SSA_green_temp+W_coef_temp_green[5]*SSA_green_temp*AAE_temp+W_coef_temp_green[6]*ln_Tr_green*AAE_temp+W_coef_temp_green[7]*ln_Tr_green*AAE_temp*SSA_green_temp)

        b_abs_blue_temp = batn_blue*(W_coef_temp_blue[0]+W_coef_temp_blue[1]*ln_Tr_blue+W_coef_temp_blue[2]*SSA_blue_temp+W_coef_temp_blue[3]*AAE_temp+W_coef_temp_blue[4]*ln_Tr_blue*SSA_blue_temp+W_coef_temp_blue[5]*SSA_blue_temp*AAE_temp+W_coef_temp_blue[6]*ln_Tr_blue*AAE_temp+W_coef_temp_blue[7]*ln_Tr_blue*AAE_temp*SSA_blue_temp)

        #create new g_term, AAE and SSA
        g_term_red_temp=b_abs_red_temp/batn_red
        g_term_green_temp=b_abs_green_temp/batn_green
        g_term_blue_temp=b_abs_blue_temp/batn_blue

        AAE_three_lambda = AAE_power_fit(b_abs_red_temp,b_abs_green_temp,b_abs_blue_temp,w1,w2,w3)

        AAE_temp = AAE_three_lambda
        AAE_temp[AAE_temp < 0 ] = 0; AAE_temp[AAE_temp > 10 ] = 0
    
        SSA_red_temp = bscat_red/(bscat_red+b_abs_red_temp)
        SSA_green_temp = bscat_green/(bscat_green+b_abs_green_temp)
        SSA_blue_temp = bscat_blue/(bscat_blue+b_abs_blue_temp)

        
        #/check if convergence is reached (tolerance is set to be 1.5%)
        valid = ~(np.isnan(AAE_previous_cycle) | np.isnan(AAE_temp))
        mydata = odr.Data(AAE_previous_cycle[valid],AAE_temp[valid])
        model = odr.Model(linear_func)
        myodr = odr.ODR(mydata, model, beta0=[0., 1.])
        output = myodr.run()
        W_coef = output.beta
        print (W_coef[0])

        file_M_2 = pd.DataFrame() 
        file_M_2['datetime'] = datetime
        file_M_2['AAE'] = AAE_temp; file_M_2['SSA_red'] = SSA_red_temp ;file_M_2['SSA_green'] = SSA_green_temp ; file_M_2['SSA_blue'] = SSA_blue_temp
        file_M_2['b_abs_red'] = b_abs_red_temp ;file_M_2['b_abs_green'] = b_abs_green_temp; file_M_2['b_abs_blue'] = b_abs_blue_temp ; 

        print(file_M_2) 

In [11]:
#Implement Module B
para_wl_1 = 652
para_wl_2 = 528
para_wl_3 = 467
para_algo = "L2020"

correct_Babs(bscat_red,bscat_green,bscat_blue,batn_red,batn_green,batn_blue,Tr_red,Tr_green,Tr_blue, para_wl_1, para_wl_2, para_wl_3, algorithm = para_algo)

c:\Users\cphal\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\arraylike.py:402: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\cphal\AppData\Local\Temp\ipykernel_11896\3032156752.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batn_red[np.isnan(AAE_temp)] = 0
C:\Users\cphal\AppData\Local\Temp\ipykernel_11896\3032156752.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batn_red[SSA_red_temp < 0 ] = np.nan;     batn_red[SSA_red_temp >1 ] = np.nan
C:\Users\cphal\AppData\Local\Temp\ipykernel_11896\3032156752.py:36: SettingWithCopy

////////////iterative process////////////
Loop No. 1
W_coef_temp_red [ 0.239035   0.34574   -0.157059  -0.0433201 -0.468967   0.0703361
 -0.565455   0.727888 ]
W_coef_temp_green [ 0.24705159  0.14315453 -0.19488072 -0.05285423 -0.23772489  0.1100104
 -0.31951781  0.40098888]
W_coef_temp_blue [ 0.29652547  0.15644998 -0.26501279 -0.09787212 -0.26733389  0.169781
 -0.24515467  0.30759007]
1.6536682868266381
                datetime       AAE   SSA_red  SSA_green  SSA_blue  b_abs_red  \
0    2018-12-01 00:00:00       NaN       NaN        NaN       NaN        NaN   
1    2018-12-01 00:05:00  2.451825  0.945471   0.935359  0.935469   0.264795   
2    2018-12-01 00:10:00  2.924283  0.960010   0.945339  0.941589   0.227455   
3    2018-12-01 00:15:00  2.482381  0.946996   0.936317  0.934525   0.335240   
4    2018-12-01 00:20:00  1.885179  0.930829   0.930757  0.933235   0.415536   
...                  ...       ...       ...        ...       ...        ...   
5755 2018-12-20 23:35:00  2.319